# KoGPT-2 Binary Classification

### 코랩 세팅

In [14]:
%cd /content/drive/MyDrive/dacon/text_label

[Errno 2] No such file or directory: '/content/drive/MyDrive/dacon/text_label'
/content


In [15]:
!pip install -U transformers sentencepiece -q

### NLP 작업 환경 설정

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler

from keras.utils import pad_sequences

from transformers import AdamW
from transformers import AutoTokenizer, AutoModel

### NumPy와 PyTorch 작업의 재현성을 보장하기 위해 무작위 시드를 설정
##### 코드를 실행한 결과가 항상 동일할 수 있도록 시드를 고정해 줍니다.

In [17]:
#이 함수는 데이터 처리와 머신러닝에서 재현성을 위해 무작위 시드를 설정하는 데 사용됩니다.
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    #시스템에 GPU(CUDA)가 있는지 확인합니다.
    if torch.cuda.is_available():
        # GPU가 사용 가능한 경우, 이 줄은 GPU의 무작위 숫자 생성기의 시드를 설정하여 GPU 가속 작업에서 재현성을 보장합니다.
        torch.cuda.manual_seed_all(seed)

set_seed()

### 시스템 가용성에 따라 GPU 또는 CPU를 사용할 것인지를 결정

In [18]:
#시스템의 가용성에 따라 'cuda:0' (GPU 사용) 또는 'cpu' (CPU 사용)로 장치를 설정합니다.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

### 데이터 불러오기

In [19]:
from google.colab import drive
drive.mount('/content/drive')

df_train = pd.read_csv('/content/drive/MyDrive/월간데이콘쇼츠/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/월간데이콘쇼츠/test.csv')

Mounted at /content/drive


### 데이터 전처리

In [20]:
#'human_reviews'와 ai_reviews 빈 리스트를 정의
human_reviews = []
ai_reviews = []

##### 클래스 균형을 맞추기 위해 가짜 데이터 중 하나만 Random Choice

In [21]:
for i, label in enumerate(df_train['label']):
  #'labels'라는 리스트를 생성, 1,2,3,4 지정
  labels = [1,2,3,4]
  #'labels' 리스트에서 현재 레코드의 레이블 값을 제거합니다. 이렇게 하면 'labels' 리스트에서 현재 레이블을 제외한 다른 레이블이 남게 됩니다.
  labels.remove(label)
  #레이블을 하나만 선택한다.
  ai_label = np.random.choice(labels)

  human_reviews.append(df_train.iloc[i, label])
  ai_reviews.append(df_train.iloc[i, ai_label])

In [22]:
# 이진 Label 0: 가짜 1: 인간
human_labels = [1] * len(human_reviews)
ai_labels = [0] * len(ai_reviews)

In [23]:
reviews = human_reviews + ai_reviews
labels = human_labels + ai_labels

### Tokenizer & Model
##### BaseLine의 사용한 KoGPT-2 사용

In [24]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

### GPT Classifier 모델 구현

In [25]:
class GPTModel(nn.Module):
  def __init__(self, num_classes=2):
    # 부모 클래스인 nn.Module의 생성자를 호출하여 클래스를 초기화합니다.
    super(GPTModel, self).__init__()
    #클래스 내에서 사용할 클래스 수를 저장하는 인스턴스 변수를 생성하고 초기화합니다.
    self.num_classes = num_classes
    # 한국어 GPT-2 모델인 'skt/kogpt2-base-v2'를 불러와서 self.gpt 변수에 할당하고, 이 모델은 사전 훈련된 모델로, 텍스트 데이터를 이해하고 생성하는 데 사용됩니다.
    self.gpt = AutoModel.from_pretrained('skt/kogpt2-base-v2')
    #bias를 False로 설정하여 편향되지 않게 합니다.
    self.fc = nn.Linear(768, num_classes, bias=False)

  #모델의 순전파(forward) 메서드를 정의합니다.
  def forward(self, input_ids, attention_mask, labels=None):
    #사전 훈련된 GPT-2 모델(self.gpt)에 입력 데이터(input_ids와 attention_mask)를 전달하여 모델의 출력을 얻습니다.
    output = self.gpt(input_ids=input_ids,
                      attention_mask=attention_mask)
    #모델의 출력을 선형 변환 레이어(self.fc)에 전달하여 최종 출력을 생성합니다.
    output = self.fc(output[0])
    # output에서 각 클래스에 해당하는 예측값을 추출합니다. 여기서 torch.arange(self.num_classes, device=device)를 사용하여 클래스 인덱스를 생성하고, -1을 사용하여 마지막 타임 스텝의 예측값을 선택합니다.
    output = output[torch.arange(self.num_classes, device=device), -1]
    return output

In [26]:
model = GPTModel()
model = model.to(device)

### Data Loader
##### input_ids, attention_mask 생성

In [27]:
# 주어진 텍스트 데이터(reviews)를 토큰화하는 과정입니다. 각 텍스트(s)는 tokenizer.tokenize() 함수를 사용하여 토큰 리스트로 변환됩니다. tokenized_texts는 이러한 토큰 리스트들의 리스트가 됩니다.
tokenized_texts = [tokenizer.tokenize(s) for s in reviews]

#토큰화된 텍스트 중 가장 긴 시퀀스의 길이를 구합니다.
MAX_LEN = max([len(x) for x in tokenized_texts])
#토큰화된 텍스트를 모델이 이해할 수 있는 정수형 시퀀스로 변환합니다. 각 토큰은 tokenizer.convert_tokens_to_ids() 함수를 사용하여 정수로 변환됩니다. 결과는 정수 시퀀스의 리스트인 input_ids로 저장됩니다.
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
#input_ids 리스트의 시퀀스를 가장 긴 시퀀스의 길이(MAX_LEN)에 맞게 패딩하여 정수 텐서로 만듭니다. 'dtype'는 데이터 타입을 나타내며, 'truncating' 및 'padding'은 시퀀스를 잘라내거나 패딩할 때의 규칙을 정의합니다. 이러한 작업은 모델이 일정한 크기의 입력을 처리할 수 있도록 합니다.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

attention_masks = []

for seq in tqdm(input_ids):
    #각 시퀀스의 어텐션 마스크를 생성합니다. 어텐션 마스크는 정수 시퀀스의 각 위치에서 0이 아닌 값에 대한 마스크를 생성합니다. 이로써 모델은 패딩된 부분을 무시하고 실제 토큰만 처리합니다.
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

train_inputs = torch.tensor(input_ids)
train_labels = torch.tensor(labels)
train_masks = torch.tensor(attention_masks)

100%|██████████| 100/100 [00:00<00:00, 7508.87it/s]


In [28]:
#미니배치 크기는 한 번에 모델에 공급되는 데이터의 샘플 수입니다.
BATCH_SIZE = 2

train_data = TensorDataset(train_inputs, train_masks, train_labels)
#무작위 샘플링을 사용하여 학습 데이터를 샘플링하는 데이터 샘플러를 생성합니다. 이것은 데이터를 무작위로 섞고 각 미니배치에 다양한 데이터가 포함되도록 합니다.
train_sampler = RandomSampler(train_data)
#PyTorch의 DataLoader를 사용하여 학습 데이터로더를 생성합니다. train_data는 학습 데이터셋, train_sampler는 데이터 샘플러, 그리고 batch_size는 미니배치 크기를 지정합니다. 이 데이터로더를 사용하면 학습 과정에서 미니배치 단위로 데이터를 모델에 공급할 수 있습니다.
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

### 학습 세팅

In [29]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
criterion = nn.CrossEntropyLoss()
epochs = 8

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
# Accuracy Metric
def flat_accuracy(preds, labels):
    #preds에서 각 예측 결과에서 가장 높은 확률을 가진 클래스의 인덱스를 추출합니다. np.argmax() 함수는 가장 큰 값의 인덱스를 반환하며, axis=1은 각 행(샘플)에서 최댓값을 찾음을 나타냅니다. 결과로 얻은 인덱스를 1차원 배열로 변환합니다.
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    #pred_flat와 labels_flat을 비교하여 정확도를 계산합니다. pred_flat == labels_flat는 각 위치에서 예측값과 실제 레이블이 일치하는지를 나타내는 부울(Boolean) 배열을 생성하고, np.sum()을 사용하여 일치하는 개수를 계산합니다. 이를 전체 샘플 수로 나누어 정확도를 계산합니다.
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

### 학습

In [31]:
for epoch in range(epochs):
    print('--------------------------------------------------------------------')
    print(f'Epoch "{epoch + 1}"')

    print('Train Mode:', end=" ")
    total_loss, train_accuracy = 0.0, 0.0
    model.train()

    # 훈련 데이터로더(train_dataloader)를 통해 미니배치(batch)를 반복합니다. 각 미니배치는 모델에 공급됩니다.
    for batch in tqdm(train_dataloader):
        #각 미니배치의 요소를 GPU(device)로 이동합니다. 이것은 모델을 가속화하기 위한 작업입니다.
        batch = tuple(t.to(device) for t in batch)
        #미니배치에서 입력 데이터, 어텐션 마스크, 및 레이블을 추출합니다.
        b_input_ids, b_input_mask, b_labels = batch
        #모델에 입력 데이터와 어텐션 마스크를 전달하여 모델의 출력(outputs)을 얻습니다.
        outputs = model(input_ids=b_input_ids,
                        attention_mask=b_input_mask)

        #손실을 계산합니다. 이 코드에서는 크로스 엔트로피 손실을 사용합니다.
        loss = criterion(outputs.view(-1, 2), b_labels)
        #모델의 출력을 확률값으로 변환한 후 logits 변수에 저장합니다.
        logits = outputs.view(-1, 2)
        #현재 미니배치의 손실을 전체 손실에 누적합니다.
        total_loss += loss.item()
        #역전파를 수행하여 그래디언트를 계산합니다.
        loss.backward()

        #logits 값을 NumPy 배열로 변환하여 CPU로 이동합니다.
        logits = logits.detach().cpu().numpy()
        #레이블 데이터를 NumPy 배열로 변환하여 CPU로 이동합니다.
        label_ids = b_labels.to('cpu').numpy()
        #현재 미니배치에서 계산된 정확도를 전체 훈련 정확도에 누적합니다.
        train_accuracy += flat_accuracy(logits, label_ids)

        #그래디언트 폭발을 방지하기 위해 모델 파라미터의 그래디언트를 클리핑(clipping)합니다.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        #옵티마이저를 사용하여 모델 파라미터를 업데이트합니다.
        optimizer.step()

    #모델의 그래디언트를 초기화합니다.
    model.zero_grad()
    #에포크 내에서의 평균 학습 손실을 계산합니다.
    avg_train_loss = total_loss / len(train_dataloader)
    #에포크 내에서의 평균 학습 정확도를 계산합니다.
    avg_train_acc = train_accuracy / len(train_dataloader)
    #위의 평균 훈련 손실과 정확도를 출력합니다.
    print(" Average training loss: {0:.2f}".format(avg_train_loss), end=" ")
    print(" Average training accuracy: {0:.2f}".format(avg_train_acc))

--------------------------------------------------------------------
Epoch "1"
Train Mode: 

100%|██████████| 50/50 [03:39<00:00,  4.38s/it]


 Average training loss: 1.40  Average training accuracy: 0.66
--------------------------------------------------------------------
Epoch "2"
Train Mode: 

100%|██████████| 50/50 [03:28<00:00,  4.17s/it]


 Average training loss: 0.84  Average training accuracy: 0.80
--------------------------------------------------------------------
Epoch "3"
Train Mode: 

...

 Average training loss: 1.10  Average training accuracy: 0.88
--------------------------------------------------------------------
Epoch "7"
Train Mode: 

100%|██████████| 50/50 [03:30<00:00,  4.20s/it]


 Average training loss: 0.26  Average training accuracy: 0.92
--------------------------------------------------------------------
Epoch "8"
Train Mode: 

100%|██████████| 50/50 [03:29<00:00,  4.20s/it]

 Average training loss: 1.76  Average training accuracy: 0.85


### 예측
##### test 데이터도 train 데이터와 동일한 프로세스로 처리
##### train 데이터와 달리 test 데이터는 4개의 문장 중 가장 인간이 작성했을 법한 문장을 예측하는 것이기 때문에 순서대로 배치
##### 4개의 문장 중 가장 score가 높은 2개 선택

In [32]:
test_reviews = []

for i in range(df_test.shape[0]):
  cols = [1,2,3,4]
  test_reviews.extend(df_test.iloc[i, cols])

In [33]:
#테스트 데이터(test_reviews)를 토큰화하는 과정입니다. 각 텍스트(s)는 tokenizer.tokenize() 함수를 사용하여 토큰 리스트로 변환됩니다. tokenized_texts는 이러한 토큰 리스트들의 리스트가 됩니다.
tokenized_texts = [tokenizer.tokenize(s) for s in test_reviews]

#토큰화된 텍스트를 모델이 이해할 수 있는 정수 시퀀스로 변환합니다. 각 토큰은 tokenizer.convert_tokens_to_ids() 함수를 사용하여 정수로 변환됩니다. 결과는 정수 시퀀스의 리스트인 input_ids로 저장됩니다.
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
#input_ids 리스트의 시퀀스를 가장 긴 시퀀스의 길이(MAX_LEN)에 맞게 패딩하여 정수 텐서로 만듭니다. 'dtype'는 데이터 타입을 나타내며, 'truncating' 및 'padding'은 시퀀스를 잘라내거나 패딩할 때의 규칙을 정의합니다. 이러한 작업은 모델이 일정한 크기의 입력을 처리할 수 있도록 합니다.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

attention_masks = []

for seq in input_ids:
    #각 시퀀스의 어텐션 마스크를 생성합니다. 어텐션 마스크는 정수 시퀀스의 각 위치에서 0이 아닌 값에 대한 마스크를 생성합니다. 이로써 모델은 패딩된 부분을 무시하고 실제 토큰만 처리합니다.
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

### 0과 1을 예측하는 것이기 아니기 때문에 확률 Score를 저장

In [34]:
#모델을 평가(Evaluation) 모드로 설정합니다. 이 모드에서는 모델이 훈련 중에는 수행하지 않는 동작을 하게 됩니다.
model.eval()

preds = []

for batch in tqdm(test_dataloader):
  #각 미니배치의 요소를 GPU(device)로 이동합니다. 이것은 모델을 가속화하기 위한 작업입니다.
  batch = tuple(t.to(device) for t in batch)
  #미니배치에서 입력 데이터와 어텐션 마스크를 추출합니다.
  b_input_ids, b_input_mask = batch
  #모델에 입력 데이터와 어텐션 마스크를 전달하여 모델의 출력(outputs)을 얻습니다.
  outputs = model(input_ids=b_input_ids,
                  attention_mask=b_input_mask)

  #모델의 출력을 확률값으로 변환한 후, logits 변수에 저장합니다. .view(-1, 2)는 출력을 2개의 클래스에 대한 확률 값으로 재구성합니다. 그런 다음 .detach()를 사용하여 그래디언트 연산을 비활성화하고, .cpu().numpy()를 사용하여 결과를 NumPy 배열로 변환합니다.
  logits = outputs.view(-1, 2).detach().cpu().numpy()
  preds.extend(logits)

preds = np.array(preds)

100%|██████████| 2200/2200 [42:23<00:00,  1.16s/it]


### 가장 높은 Score를 가진 2 문장 label 선택

In [35]:
pred_labels = []

for i in range(0,preds.shape[0],4):
  tmp = preds[i:i+4,1] # 4 문장씩 검사
  label1 = np.argmax(tmp) # 가장 높은 score
  tmp[label1] = -np.inf
  label2 = np.argmax(tmp) # 두번째로 높은 score

  label = str(label1 + 1) + str(label2 + 1)
  pred_labels.append(label)

### 제출

In [36]:
submit = pd.read_csv('/content/drive/MyDrive/월간데이콘쇼츠/sample_submission.csv')
submit['label'] = pred_labels
submit

,id,label
0,TEST_0000,34
1,TEST_0001,12
2,TEST_0002,24
3,TEST_0003,23
4,TEST_0004,32
...,...,...
1095,TEST_1095,24
1096,TEST_1096,24
1097,TEST_1097,41
1098,TEST_1098,43
